# Budget Analysis for SR methods

The analysis has the following parts: 

1. Maximum accuracy obtained per noise level
1. Basic learning curve 
1. Cost of annotation of the learning curve 

## Steps

1. Compute maximum accuracy for dataset
1. Compute maximum accuracy for fist-1
1. Compute levels of performance
1. Compute budget locations



In [3]:
DATA = 'C:/Users/mramire8/Documents/Research/Oracle confidence and Interruption/dataset/aclImdb/raw-data'
import sys
import os
sys.path.append(os.path.abspath('C:/cygwin/home/mramire8/python_code/structured/'))
sys.path.append(os.path.abspath('/Users/maru/MyCode/structured/'))

import learner

from utilities.datautils import load_dataset
import utilities.experimentutils as exputil
import numpy as np
import experiment.base as exp
import nltk


# EXample of IMDB 
kvalues = [10, 25, 50, 75, 100]
kv = [ 25, 50, 75, 100]
cost = np.array([5.7, 8.2, 10.9, 15.9, 16.7])

def fn_int(x, cost, kvalues):
    
    binx = min(np.digitize([x], kvalues)[0], len(cost)-1)
    lbbinx = max(binx-1, 0) 
#     print binx, lbbinx
    y1 = cost[lbbinx] if lbbinx>=0  else 0
    y2 = cost[binx]
    x1 = kvalues[lbbinx] if lbbinx >=0 else 0
    x2 = kvalues[binx]
#     print x1,x2,y1,y2
    
    m = (y2-y1) / (x2-x1)
    b = y2 - m * x2
    
    if x < kvalues[0]:
        y = cost[0]
    elif x > kvalues[-1] :
        y = cost[-1]
    else:
        y = (m * x) + b
    return y

print fn_int(98, cost, kvalues)

16.636


In [5]:
## Get the data ready
imdb_path = 'C:/Users/mramire8/Documents/Research/Oracle confidence and Interruption/dataset/aclImdb/raw-data'
# imdb_path = '/Users/maru/MyCode/data/imdb'

vct = exputil.get_vectorizer({'vectorizer':'bow', 'limit':None, 'min_size':2})

imdb =  load_dataset("imdb",imdb_path, keep_subject=True)

# sraa = load_dataset("aviation", 100, categories[0], vct2, 100, raw=True,  percent=.5, keep_subject=True)
imdb.train.bow = vct.fit_transform(imdb.train.data)
imdb.test.bow = vct.transform(imdb.test.data)




In [6]:
imdb.test.bow = vct.transform(imdb.test.data)

expert_config = {'type'           : "noisyreluctantscale", 
    'model'          : "lrl1",
    'parameter'      : 0.3,
    'costmodel'      : 1,
    'sent_tokenizer' : "first1snippet",
    'snip_size'      : (1,1),
    'threshold'      : .4,
    'scale': 0.
}
imdb_expert = exputil.get_expert(expert_config, size=len(imdb.train.data))
imdb_expert.fit(imdb.train.bow, y=imdb.train.target, vct=vct)


AttributeError: lower not found

In [ ]:
# for every document, get the snippet, convert to bow, 
# get the label from expert
# train a student classifier on resulting dataset
# test levels 

In [ ]:
# create dataset
# test student 

In [ ]:
# Create the data for the labels 
def add_instance(mat, to_add):
    test_sent = mat
    if len(to_add) == 0: #.shape[0] == 0:
        return mat
    
    for instance in to_add:
        if isinstance(test_sent, list):
            test_sent = instance
        else:
            test_sent = vstack([test_sent, instance], format='csr')
    return test_sent

def convert2mat(tx_lst, target):
    tx = []
    ty = []
    for x,y in zip(txt_lst, target):
        tx.extend(x)
        ty.extend([y]*len(x))
    return np.array(tx), np.array(ty)

def create_data(org_data, expert,  snip_tk, vct):
    snips = snip_tk.tokenize_sents(org_data.data)
    tx, y = convert2mat(snips, org_data.target)
    vecx = vct.transform(tx)
    pred = expert.label({'bow':vecx,'snippet':tx, 'index':range(len(tx))}, y=y)
    return tx, vecx, pred




In [ ]:
from sklearn import cross_validation

def test_level(org_data, expert, vct, clf)
    rnd_state = np.random.RandomState(123)
    train_sizes=np.linspace(.1, 1.0, 5)

    tk = exputil.get_tokenizer('first1snippet',snip_size=(s,s))
    cv = cross_validation.ShuffleSplit(n, n_iter=5, test_size=0.0,
                                       random_state=rnd_state)
    tx, bow, pred = create_data(org_data.train, expert, tk, vct)
    sizes = np.array([len(x.split()) for x in tx])
    cost = np.array([fn_int(x, cost, kvalues) for x in sizes])
    
    results = []
    for train_index, _ in cv:
        
test_level(imdb, )